# 2 Layer Neural Network

## Imports

In [1]:
import os
import pandas as pd

## Import data and create .csv

In [2]:
# Directory to MRI Images
base_dir = '/Users/benrandoing/Downloads/archive/AugmentedAlzheimerDataset'
classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

if not os.path.exists('dataset.csv'):
    data = []

    for label, class_name in enumerate(classes):
        class_dir = os.path.join(base_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            data.append([image_path, label])

    df = pd.DataFrame(data, columns=['path', 'label'])
    df.to_csv('dataset.csv', index=False)

## Data Loading and Pre-Processing

In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.dataframe = pd.read_csv(csv_file)
        self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)  # Shuffle the dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = int(self.dataframe.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224 X 224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load the entire dataset
dataset = CustomDataset(csv_file='dataset.csv', transform=transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [4]:
print(f'Training Samples: {len(train_dataset)}')
print(f'Validation Samples: {len(val_dataset)}')
print(f'Test Samples: {len(test_dataset)}')

Training Samples: 23788
Validation Samples: 5097
Test Samples: 5099


In [5]:
print(train_dataset)

In [6]:
import matplotlib.pyplot as plt
#plt.plot(train_dataset[0][0])
#plt.show()

## Pytorch Logistic Regression Model

In [7]:
class FiveLayerNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim):
        super(FiveLayerNN, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim1)
        self.fc2 = torch.nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = torch.nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = torch.nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = torch.nn.Linear(hidden_dim4, output_dim)

    def forward(self, x):
        #x = x.view(-1, 224 * 224 * 3)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

input_dim = 224 * 224 * 3  # Assuming the images are RGB
hidden_dim1 = 512
hidden_dim2 = 256
hidden_dim3 = 128
hidden_dim4 = 64
output_dim = 4  # Four classes

#model = FiveLayerNN(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim)
import torchvision
from torchvision.models import resnet18, ResNet18_Weights
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

model.fc = torch.nn.Linear(model.fc.in_features, output_dim)


## Training Loop

In [8]:
learning_rate = 0.001
num_epochs = 20
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()

import torch.optim
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    i = 0
    for inputs, labels in train_loader:
        i = i + 1
        print("Training continues..", total_loss, "current i = ", i)    
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    #lr decay
    #lr_scheduler.step()

    print(f"Iteration {epoch} - Loss = {total_loss / len(train_loader)}")

Training continues.. 0.0 current i =  1
Training continues.. 1.4263499975204468 current i =  2
Training continues.. 5.120266556739807 current i =  3
Training continues.. 8.428125977516174 current i =  4
Training continues.. 10.543232798576355 current i =  5
Training continues.. 12.810245633125305 current i =  6
Training continues.. 14.947182774543762 current i =  7
Training continues.. 16.718878746032715 current i =  8
Training continues.. 18.46541678905487 current i =  9
Training continues.. 19.88349986076355 current i =  10
Training continues.. 21.31885039806366 current i =  11
Training continues.. 22.89302897453308 current i =  12
Training continues.. 24.299073576927185 current i =  13
Training continues.. 25.665361642837524 current i =  14
Training continues.. 27.026344180107117 current i =  15
Training continues.. 28.625990986824036 current i =  16
Training continues.. 29.91148614883423 current i =  17
Training continues.. 31.349323630332947 current i =  18
Training continues.. 32

Training continues.. 183.3124237060547 current i =  149
Training continues.. 184.1912700533867 current i =  150
Training continues.. 185.0742319226265 current i =  151
Training continues.. 185.83030462265015 current i =  152
Training continues.. 186.66683346033096 current i =  153
Training continues.. 187.74933570623398 current i =  154
Training continues.. 188.72585892677307 current i =  155
Training continues.. 189.67435824871063 current i =  156
Training continues.. 190.4920238852501 current i =  157
Training continues.. 191.34994304180145 current i =  158
Training continues.. 192.24146115779877 current i =  159
Training continues.. 193.18362218141556 current i =  160
Training continues.. 193.91529697179794 current i =  161
Training continues.. 194.75204783678055 current i =  162
Training continues.. 195.5033985376358 current i =  163
Training continues.. 196.37418788671494 current i =  164
Training continues.. 197.19197416305542 current i =  165
Training continues.. 197.88320571184

Training continues.. 293.7495861053467 current i =  294
Training continues.. 294.48993796110153 current i =  295
Training continues.. 295.14964908361435 current i =  296
Training continues.. 295.76019990444183 current i =  297
Training continues.. 296.57848542928696 current i =  298
Training continues.. 297.3032215833664 current i =  299
Training continues.. 297.9525447487831 current i =  300
Training continues.. 298.73300701379776 current i =  301
Training continues.. 299.307635307312 current i =  302
Training continues.. 299.94731307029724 current i =  303
Training continues.. 300.61159855127335 current i =  304
Training continues.. 301.2545365691185 current i =  305
Training continues.. 301.79890835285187 current i =  306
Training continues.. 302.4129822254181 current i =  307
Training continues.. 303.0713435411453 current i =  308
Training continues.. 303.7832933664322 current i =  309
Training continues.. 304.5155639052391 current i =  310
Training continues.. 305.27224987745285 c

Training continues.. 39.36983558535576 current i =  69
Training continues.. 39.80686354637146 current i =  70
Training continues.. 40.23073253035545 current i =  71
Training continues.. 40.67722341418266 current i =  72
Training continues.. 41.22879698872566 current i =  73
Training continues.. 41.916723161935806 current i =  74
Training continues.. 42.74812349677086 current i =  75
Training continues.. 43.442818373441696 current i =  76
Training continues.. 44.3321273624897 current i =  77
Training continues.. 44.98985508084297 current i =  78
Training continues.. 45.601724058389664 current i =  79
Training continues.. 46.10538312792778 current i =  80
Training continues.. 46.56560042500496 current i =  81
Training continues.. 47.22177121043205 current i =  82
Training continues.. 47.85394695401192 current i =  83
Training continues.. 48.31217882037163 current i =  84
Training continues.. 48.79040586948395 current i =  85
Training continues.. 49.44301116466522 current i =  86
Training

Training continues.. 113.29497250914574 current i =  216
Training continues.. 113.91010156273842 current i =  217
Training continues.. 114.36842009425163 current i =  218
Training continues.. 114.68945625424385 current i =  219
Training continues.. 115.25883111357689 current i =  220
Training continues.. 115.65998527407646 current i =  221
Training continues.. 116.17230948805809 current i =  222
Training continues.. 116.48448231816292 current i =  223
Training continues.. 116.88683071732521 current i =  224
Training continues.. 117.31376591324806 current i =  225
Training continues.. 117.77589213848114 current i =  226
Training continues.. 118.11076191067696 current i =  227
Training continues.. 118.54855480790138 current i =  228
Training continues.. 118.94482830166817 current i =  229
Training continues.. 119.23791888356209 current i =  230
Training continues.. 119.62017717957497 current i =  231
Training continues.. 120.04772049188614 current i =  232
Training continues.. 120.607810

...

Training continues.. 0.5486872952315025 current i =  31
Training continues.. 0.5499161485931836 current i =  32
Training continues.. 0.5555423944606446 current i =  33
Training continues.. 0.556746621907223 current i =  34
Training continues.. 0.5723118123714812 current i =  35
Training continues.. 0.5731862623360939 current i =  36
Training continues.. 0.5743386832182296 current i =  37
Training continues.. 0.5749998427927494 current i =  38
Training continues.. 0.6397645957767963 current i =  39
Training continues.. 0.6399410893500317 current i =  40
Training continues.. 0.6403876226977445 current i =  41
Training continues.. 0.6533540962846018 current i =  42
Training continues.. 0.6654265849501826 current i =  43
Training continues.. 0.8265238283784129 current i =  44
Training continues.. 0.870049747696612 current i =  45
Training continues.. 0.8892475655884482 current i =  46
Training continues.. 0.8913871660479344 current i =  47
Training continues.. 0.9048643685528077 current i 

Training continues.. 3.593134538255981 current i =  177
Training continues.. 3.616662590749911 current i =  178
Training continues.. 3.639688522540382 current i =  179
Training continues.. 3.657618443467072 current i =  180
Training continues.. 3.658604520125664 current i =  181
Training continues.. 3.6613655909750378 current i =  182
Training continues.. 3.6628846725070616 current i =  183
Training continues.. 3.6633016003033845 current i =  184
Training continues.. 3.667290141034755 current i =  185
Training continues.. 3.6681252312118886 current i =  186
Training continues.. 3.677032181454706 current i =  187
Training continues.. 3.6929664941708324 current i =  188
Training continues.. 3.694273973509553 current i =  189
Training continues.. 3.6947099046810763 current i =  190
Training continues.. 3.6998307627654867 current i =  191
Training continues.. 3.700094034909853 current i =  192
Training continues.. 3.700528986999416 current i =  193
Training continues.. 3.7032922313228482 c

Training continues.. 6.430241360532818 current i =  323
Training continues.. 6.432762047479628 current i =  324
Training continues.. 6.434516143606743 current i =  325
Training continues.. 6.434941667248495 current i =  326
Training continues.. 6.435155969375046 current i =  327
Training continues.. 6.441558828373672 current i =  328
Training continues.. 6.485958019067766 current i =  329
Training continues.. 6.486834952287609 current i =  330
Training continues.. 6.510008379496867 current i =  331
Training continues.. 6.574142291821772 current i =  332
Training continues.. 6.601043777220184 current i =  333
Training continues.. 6.613192840159172 current i =  334
Training continues.. 6.621246779017383 current i =  335
Training continues.. 6.635861224174732 current i =  336
Training continues.. 6.639712333824718 current i =  337
Training continues.. 6.642374295654008 current i =  338
Training continues.. 6.6429061491217 current i =  339
Training continues.. 6.643271245062351 current i =

Training continues.. 0.729612599232496 current i =  97
Training continues.. 0.7315021637950849 current i =  98
Training continues.. 0.7390712035012257 current i =  99
Training continues.. 0.7391954472004727 current i =  100
Training continues.. 0.7407754545747594 current i =  101
Training continues.. 0.7418876452320546 current i =  102
Training continues.. 0.7424316409342282 current i =  103
Training continues.. 0.743109127466596 current i =  104
Training continues.. 0.7431842218102247 current i =  105
Training continues.. 0.7465991672543169 current i =  106
Training continues.. 0.7468944188331079 current i =  107
Training continues.. 0.7484091733240348 current i =  108
Training continues.. 0.7491782224860799 current i =  109
Training continues.. 0.7506161926903587 current i =  110
Training continues.. 0.7511188463431608 current i =  111
Training continues.. 0.7536540924229485 current i =  112
Training continues.. 0.7548448369961989 current i =  113
Training continues.. 0.7549867877569

Training continues.. 3.447521653248259 current i =  242
Training continues.. 3.450033449789771 current i =  243
Training continues.. 3.450611865544488 current i =  244
Training continues.. 3.4513284566237417 current i =  245
Training continues.. 3.4630255589618173 current i =  246
Training continues.. 3.51078940950174 current i =  247
Training continues.. 3.51323144509297 current i =  248
Training continues.. 3.5136956148671743 current i =  249
Training continues.. 3.5268176109420892 current i =  250
Training continues.. 3.5297945216188964 current i =  251
Training continues.. 3.5309143738013518 current i =  252
Training continues.. 3.5419420470607292 current i =  253
Training continues.. 3.549685055961163 current i =  254
Training continues.. 3.594088608970196 current i =  255
Training continues.. 3.5946458704893303 current i =  256
Training continues.. 3.6115246467416 current i =  257
Training continues.. 3.7087630532805633 current i =  258
Training continues.. 3.710144296859653 curr

## Evaluation

In [9]:
# Validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

validation_accuracy = 100 * correct / total
print(f"Validation Accuracy: {validation_accuracy}%")

Validation Accuracy: 91.81871689228959%


In [10]:
# Testing loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy}%")

Test Accuracy: 91.09629339086095%


In [1]:
import torch
torch.save(model.state_dict(), 'resnet18_more_epochs_rmsprop_model_weights.pth')

NameError: name 'model' is not defined